In [29]:
import pickle
# from sympy import symbols, expand, lambdify, Expr, simplify, pprint, Rel
import sympy as sp
import math
from pyomo.environ import *
import pyomo.environ as pyo
from typing import List
import itertools
import numpy as np
from IPython.display import display

In [30]:
def affine_expr(coeffs, symbols):
    return sum(c * s for c, s in zip(coeffs[:-1], symbols)) + coeffs[-1]

In [31]:
def generate_region_combos(region_sizes, n_gl):
    """Generate region index combinations based on critical region structure."""
    n_theta = len(region_sizes)
    region_combo_shape = []
    for k in range(n_theta):
        n_paths = int(np.prod(n_gl[:k])) if k > 0 else 1
        region_combo_shape.extend([range(region_sizes[k])] * n_paths)
    return list(itertools.product(*region_combo_shape))

In [32]:
def is_same_inequality(a, b):
    return sp.simplify(a.lhs - b.lhs) == 0 and type(a) == type(b)

def normalized_lhs(ineq):
    return ineq.lhs.expand()

In [33]:
def compute_sf_expressions_region_combo_based_fixed(
    theta_bounds_list,
    theta_regions_list,
    joint_pdf_expr,
    d_syms,
    n_gl_list,
    theta_syms
):
    n_theta = len(theta_syms)
    quad_data = [np.polynomial.legendre.leggauss(n) for n in n_gl_list]
    region_sizes = [bounds.shape[0] for bounds in theta_bounds_list]
    region_combos = generate_region_combos(region_sizes, n_gl_list)

    sf_exprs = []
    sf_regions = []

    for region_combo in region_combos:
        combo_ptr = 0
        # Initialize integration paths: (theta_vals, weight, scale, constraints)
        paths = [([], 1, 1, [])]

        for level in range(n_theta):
            xi, wi = quad_data[level]
            new_paths = []

            for theta_vals, weight, scale, constraints in paths:
                region_idx = region_combo[combo_ptr]
                combo_ptr += 1

                bound_inputs = theta_vals + list(d_syms)
                bounds = theta_bounds_list[level][region_idx]
                t_min = affine_expr(bounds[0], bound_inputs)
                t_max = affine_expr(bounds[1], bound_inputs)

                # Get level-specific region constraints
                rows = theta_regions_list[level][region_idx]
                level_constraints = []
                for row in rows:
                    t_coeffs = row[:level]
                    d_coeffs = row[level:-1]
                    const = row[-1]
                    lhs = sum(c * theta_vals[i] for i, c in enumerate(t_coeffs)) + \
                          sum(c * d for c, d in zip(d_coeffs, d_syms)) + const
                    # level_constraints.append(sp.simplify(lhs <= 0))
                    level_constraints.append(lhs <= 0)

                new_constraints = constraints + level_constraints

                # Quadrature expansion for this level
                for q in range(len(xi)):
                    t = 0.5 * (t_max - t_min) * xi[q] + 0.5 * (t_max + t_min)
                    # new_theta_vals = theta_vals + [sp.simplify(t)]
                    new_theta_vals = theta_vals + [t]
                    new_weight = weight * wi[q]
                    new_scale = scale * 0.5 * (t_max - t_min)
                    new_paths.append((new_theta_vals, new_weight, new_scale, new_constraints))

            paths = new_paths

        # Final integration and region collection
        sf_sum = 0
        all_constraints = []
        for theta_vals, weight, scale, constraints in paths:
            theta_subs = {sym: val for sym, val in zip(theta_syms, theta_vals)}
            pdf_val = joint_pdf_expr.subs(theta_subs)
            sf_sum += weight * scale * pdf_val
            all_constraints.extend(constraints)

        # Deduplicate constraints symbolically
        unique_constraints = []
        for c in all_constraints:
            if not any(normalized_lhs(c) == normalized_lhs(u) and type(c) == type(u) for u in unique_constraints):
                unique_constraints.append(c)  # keep original form for clarity
            # if not any(is_same_inequality(c, u) for u in unique_constraints):
            #     unique_constraints.append(sp.simplify(c))

        sf_exprs.append(sf_sum)
        # sf_regions.append(sorted(all_constraints, key=str))
        # sf_exprs.append(sp.simplify(sf_sum))
        sf_regions.append(sorted(unique_constraints, key=str))

    return sf_exprs, sf_regions

In [34]:
def compute_all_sf_expressions_generalized(
        theta_bounds_list,  # List of shape (n_regions_i, 2, coeff_len) for each θᵢ
        theta_regions_list,  # List of region constraint matrices for each θᵢ
        joint_pdf_expr,  # sympy expression in θ₁...θₙ
        d_syms,  # sympy symbols for uncertain parameters
        n_gl_list,  # List of quadrature points per θᵢ
        theta_syms  # [θ₁, θ₂, ..., θₙ]
):

        n_theta = len(theta_syms)
        assert len(theta_bounds_list) == n_theta
        assert len(theta_regions_list) == n_theta
        assert len(n_gl_list) == n_theta

        quadrature_data = [np.polynomial.legendre.leggauss(n) for n in n_gl_list]
        sf_exprs = []
        sf_regions = []

        # First loop over θ₁ regions
        n_t1_regions = theta_bounds_list[0].shape[0]
        n_q1 = n_gl_list[0]
        xi_1, wi_1 = quadrature_data[0]

        for t1_region_idx in range(n_t1_regions):
            # θ₁ bounds (only depend on d_syms)
            t1_min_expr = affine_expr(theta_bounds_list[0][t1_region_idx, 0], d_syms)
            t1_max_expr = affine_expr(theta_bounds_list[0][t1_region_idx, 1], d_syms)

            # θ₁ quadrature points
            # theta1_points = [
            #     sp.simplify(0.5 * (t1_max_expr - t1_min_expr) * xi + 0.5 * (t1_max_expr + t1_min_expr))
            #     for xi in xi_1
            # ]
            theta1_points = [
                0.5 * (t1_max_expr - t1_min_expr) * xi + 0.5 * (t1_max_expr + t1_min_expr) for xi in xi_1
            ]
            # θ₁ region constraints
            t1_region_constraints = theta_regions_list[0][t1_region_idx]
            t1_ineqs = [
                sum(c * d for c, d in zip(row[:-1], d_syms)) + row[-1] <= 0
                for row in t1_region_constraints
            ]

            # For each θ₁ point, now we consider all combinations of θ₂ region indices (per θ₁ point)
            n_t2_regions = theta_bounds_list[1].shape[0]
            t2_region_choices = list(itertools.product(range(n_t2_regions), repeat=n_q1))

            for t2_combo in t2_region_choices:
                sf_sum = 0
                region_ineqs = list(t1_ineqs)

                for i in range(n_q1):
                    t1 = theta1_points[i]
                    w1 = wi_1[i]
                    t2_region_idx = t2_combo[i]

                    # θ₂ bounds depend on θ₁ and d
                    t2_min_expr = affine_expr(theta_bounds_list[1][t2_region_idx, 0], [t1] + list(d_syms))
                    t2_max_expr = affine_expr(theta_bounds_list[1][t2_region_idx, 1], [t1] + list(d_syms))
                    xi_2, wi_2 = quadrature_data[1]
                    n_q2 = n_gl_list[1]

                    # θ₂ region constraints
                    for row in theta_regions_list[1][t2_region_idx]:
                        θ1_coeff = row[0]
                        d_coeffs = row[1:-1]
                        const = row[-1]
                        ineq = θ1_coeff * t1 + sum(c * d for c, d in zip(d_coeffs, d_syms)) + const <= 0
                        # region_ineqs.append(sp.simplify(ineq))
                        region_ineqs.append(ineq)

                    for j in range(n_q2):
                        t2 = 0.5 * (t2_max_expr - t2_min_expr) * xi_2[j] + 0.5 * (t2_max_expr + t2_min_expr)
                        # pdf_val = joint_pdf_expr.subs({theta_syms[0]: t1, theta_syms[1]: sp.simplify(t2)})
                        pdf_val = joint_pdf_expr.subs({theta_syms[0]: t1, theta_syms[1]: t2})
                        weight = w1 * wi_2[j]
                        scale = 0.25 * (t1_max_expr - t1_min_expr) * (t2_max_expr - t2_min_expr)
                        sf_sum += weight * scale * pdf_val

                # sf_exprs.append(sp.simplify(sf_sum))
                sf_exprs.append(sf_sum)
                sf_regions.append(region_ineqs)

        return sf_exprs, sf_regions

In [35]:
def generate_constraints_from_expressions(
    expr_list: List[sp.Expr],
    region_list: List[List[sp.Expr]],
    instance: ConcreteModel,
    bounds_dict:dict,
    target: float = 1.0,
    big_m: float = 1e3
):
    """
    Adds constraint expressions to a Pyomo model using Big-M logic. For each expression in expr_list,
    the constraint is enforced only if the design variables lie in the corresponding critical region
    defined in region_list.

    Args:
        expr_list: List of SymPy expressions representing SF constraints.
        region_list: List of lists of SymPy inequality expressions defining valid regions.
        instance: Pyomo model (ConcreteModel).
        target: The minimum acceptable value for each SF expression.
        big_m: Big-M constant for constraint activation.
    """
    assert len(expr_list) == len(region_list), "Each expression must have a corresponding region definition."

    if not hasattr(instance, "generated_constraints"):
        instance.generated_constraints = ConstraintList()
    if not hasattr(instance, "region_constraints"):
        instance.region_constraints = ConstraintList()
    if not hasattr(instance, "region_binaries"):
        instance.region_binaries = Var(range(len(expr_list)), within=Binary)

    sf_expr_pyomo_list = list()

    for i, (sf_expr, region_exprs) in enumerate(zip(expr_list, region_list)):
        # Get all symbols in SF expression and region inequalities
        all_syms = sf_expr.free_symbols.union(*[reg.free_symbols for reg in region_exprs])
        all_syms = list(all_syms)

        # Ensure all symbols are added to the Pyomo model
        pyomo_vars = []
        for sym in all_syms:
            var_name = str(sym)
            if not hasattr(instance, var_name):
                setattr(instance, var_name, Var(bounds=bounds_dict[var_name]))
            pyomo_vars.append(getattr(instance, var_name))

        # Create a dict for substitution and lambdify
        sym_to_pyomo = {str(sym): getattr(instance, str(sym)) for sym in all_syms}
        lambdify_vars = list(sym_to_pyomo.keys())
        lambdify_vals = [sym_to_pyomo[s] for s in lambdify_vars]

        # Lambdify SF expression
        sf_func = sp.lambdify(lambdify_vars, sf_expr, modules=[{'exp': pyo.exp, 'pi': math.pi}, 'sympy'])
        sf_pyomo = sf_func(*lambdify_vals)
        sf_expr_pyomo_list.append(sf_pyomo)

        # Constraint: enforce SF ≥ target only when region binary = 1
        instance.generated_constraints.add(
            sf_pyomo >= target - big_m * (1 - instance.region_binaries[i])
        )

        # Region constraints: region_expr <= 0 + M*(1 - delta_i)
        for reg_expr in region_exprs:
            if not isinstance(reg_expr, sp.Rel):
                raise ValueError(f"Invalid region expression: {reg_expr} is not a relational (inequality) expression.")

            reg_func_lhs = sp.lambdify(lambdify_vars, reg_expr.lhs, modules='sympy')
            reg_func_rhs = sp.lambdify(lambdify_vars, reg_expr.rhs, modules='sympy')
            lhs_pyomo = reg_func_lhs(*lambdify_vals)
            rhs_pyomo = reg_func_rhs(*lambdify_vals)

            delta = instance.region_binaries[i]
            M_term = big_m * (1 - delta)

            if reg_expr.rel_op == '<=':
                instance.region_constraints.add(lhs_pyomo <= rhs_pyomo + M_term)
            elif reg_expr.rel_op == '<':
                instance.region_constraints.add(lhs_pyomo <= rhs_pyomo - 1e-6 + M_term)
            elif reg_expr.rel_op == '>=':
                instance.region_constraints.add(lhs_pyomo >= rhs_pyomo - M_term)
            elif reg_expr.rel_op == '>':
                instance.region_constraints.add(lhs_pyomo >= rhs_pyomo + 1e-6 - M_term)
            elif reg_expr.rel_op == '==':
                instance.region_constraints.add(lhs_pyomo >= rhs_pyomo - M_term)
                instance.region_constraints.add(lhs_pyomo <= rhs_pyomo + M_term)
            else:
                raise NotImplementedError(f"Unsupported relational operator: {reg_expr.rel_op}")

    # Optional: Only one region can be active
    instance.region_exclusivity = Constraint(expr=sum(instance.region_binaries[i] for i in range(len(expr_list))) == 1)
    if not hasattr(instance, 'sf'):
        instance.sf = Var(within=NonNegativeReals)

    if not hasattr(instance, 'sf_con'):
        instance.sf_con = Constraint(expr = instance.sf == sum(sf_expr_pyomo_list[i] * instance.region_binaries[i] for i in range(len(expr_list))))

In [36]:
# with open('factored_sf_exprs.pkl', 'rb') as file:
#     factored_sf_exprs = pickle.load(file)

In [37]:
# --- Define Symbols ---
theta_syms = sp.symbols('theta_1 theta_2')
d_syms = sp.symbols('d0 d1')
theta_1, theta_2 = theta_syms
d1, d2 = d_syms

n_gl_list = [8, 8]

# Bounds arrays
theta1_bounds_array = np.array([
    [[0.0, 0.0, 0.0], [0.0, 0.0, 4.0]],  # d0 ± 10
    [[0.75, -1.5, -1.25], [0.0, 0.0, 4.0]]  # d1 ± 5
])

theta2_bounds_array = np.array([
    [[-8/3, 2.0, -4.0, 2/3], [0.0, 0.0, 0.0, 4.0]],
    [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0]],
    [[1/3, -0.5, 0.5, -1/3], [0.0, 0.0, 0.0, 4.0]]
])

# PDF
joint_pdf_expr = (2/sp.pi) * sp.exp(-2 * ((theta_1 - 2) ** 2 + (theta_2 - 2) ** 2))

# Critical regions (CRs)
# theta1_critical_regions: constraints in terms of d (shape: n_regions × n_ineqs × (m+1))
theta1_critical_regions = np.array([
    [[1, -2, -5/3]],
    [[-1, 2, 5/3]]
], dtype=object)

# theta2_critical_regions: constraints in terms of [theta1, d0, d1, const]
theta2_critical_regions = np.array([
    [[-8/3, 2, -4, -10/3], [8/3, -2, 4, -2/3]],
    [[8/3, -4.0, 4.0, -8/3], [-8/3, 2, -4, 2/3]],
    [[-8/3, 4, -4, 8/3]]
], dtype=object)

# --- Setup lists ---
theta_bounds_list = [theta1_bounds_array, theta2_bounds_array]
theta_regions_list = [theta1_critical_regions, theta2_critical_regions]

In [38]:
# sf_exprs1, sf_regions1 = compute_all_sf_expressions_generalized(
#         theta_bounds_list = theta_bounds_list, theta_regions_list=theta_regions_list, joint_pdf_expr=joint_pdf_expr, d_syms=d_syms, n_gl_list=n_gl_list,
#         theta_syms = list(theta_syms)
# )

In [39]:
sf_exprs2, sf_regions2 =  compute_sf_expressions_region_combo_based_fixed(theta_bounds_list=theta_bounds_list, theta_regions_list=theta_regions_list,
                                                                              joint_pdf_expr=joint_pdf_expr, d_syms=[d1, d2], n_gl_list=n_gl_list,theta_syms=theta_syms)

In [40]:
# print(f'Number of SF expressions: {len(sf_exprs1)}')
# print(f'Number of critical regions: {len(sf_regions1)}')

In [41]:
print(f'Number of SF expressions: {len(sf_exprs2)}')
print(f'Number of critical regions: {len(sf_regions2)}')

Number of SF expressions: 13122
Number of critical regions: 13122


In [42]:
# def compare_sf_outputs(sf_exprs1, sf_regions1, sf_exprs2, sf_regions2, simplify=True, tol=1e-8):
#     """
#     Compares two sets of SF expressions and their associated region constraints.
#     
#     Parameters:
#     - sf_exprs1, sf_exprs2: list of sympy expressions
#     - sf_regions1, sf_regions2: list of lists of sympy inequalities
#     - simplify: whether to simplify expressions before comparison
#     - tol: numerical tolerance (applied if expressions are numeric)
#     
#     Returns:
#     - report: list of tuples (index, expr_match, region_match, expr_diff)
#     - summary: dict of totals
#     """
#     n1 = len(sf_exprs1)
#     n2 = len(sf_exprs2)
#     
#     if n1 != n2:
#         print(f"Length mismatch: {n1} vs {n2}")
#         return None, None
# 
#     report = []
#     num_expr_mismatch = 0
#     num_region_mismatch = 0
# 
#     for i, (e1, r1, e2, r2) in enumerate(zip(sf_exprs1, sf_regions1, sf_exprs2, sf_regions2)):
#         if simplify:
#             e1_s = sp.simplify(e1)
#             e2_s = sp.simplify(e2)
#         else:
#             e1_s = e1
#             e2_s = e2
# 
#         # Check expression equality
#         expr_match = sp.simplify(e1_s - e2_s) == 0
# 
#         # If not symbolic match, try numerical
#         if not expr_match:
#             try:
#                 num_diff = float(sp.N(e1_s - e2_s))
#                 expr_match = abs(num_diff) < tol
#             except Exception:
#                 num_diff = None
#         else:
#             num_diff = 0
# 
#         # Check region constraints
#         r1_s = sorted([sp.simplify(c) for c in r1], key=str)
#         r2_s = sorted([sp.simplify(c) for c in r2], key=str)
#         region_match = r1_s == r2_s
# 
#         if not expr_match:
#             num_expr_mismatch += 1
#         if not region_match:
#             num_region_mismatch += 1
# 
#         report.append((i, expr_match, region_match, num_diff))
# 
#     summary = {
#         "total": n1,
#         "expr_mismatch": num_expr_mismatch,
#         "region_mismatch": num_region_mismatch,
#         "all_match": (num_expr_mismatch == 0 and num_region_mismatch == 0)
#     }
# 
#     return report, summary


In [43]:
# report, summary = compare_sf_outputs(sf_exprs1, sf_regions1, sf_exprs2, sf_regions2)

In [44]:
# print(summary)

In [45]:
# # To inspect mismatches:
# for idx, match_expr, match_region, diff in report:
#     if not (match_expr and match_region):
#         print(f"\nMismatch at index {idx}:")
#         print(f"  Expr Match: {match_expr}, Region Match: {match_region}, Expr Diff: {diff}")

In [46]:
# for i, ex in enumerate(sf_exprs):
#     print(f'SF expression {i}')
#     display(ex)
#     print('\n')

In [47]:
# for i, region_exprs in enumerate(sf_regions):
#     print(f'For region {i}:')
#     for expr in region_exprs:
#         display(expr)
#     print('\n')
# # sf_regions

In [48]:
design_bounds = {
    'd0': (0,5),
    'd1':(0,5)
}

In [49]:
m = ConcreteModel()

In [50]:
generate_constraints_from_expressions(expr_list=sf_exprs2, region_list=sf_regions2, instance=m, bounds_dict=design_bounds, target=0.8)

In [51]:
m.obj = Objective(expr=-10*m.d0 + 10*m.d1, sense=minimize)

In [52]:
results = SolverFactory('gams', solver='baron').solve(m, tee=True)

--- Job model.gms Start 07/10/25 22:13:36 45.7.0 64fbf3ce WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\45\gmsprmNT.txt
--- GAMS Parameters defined
    Input C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpcszftja5\model.gms
    Output C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpcszftja5\output.lst
    ScrDir C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpcszftja5\225a\
    SysDir C:\GAMS\45\
    CurDir C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpcszftja5\
    LogOption 3
Licensee: MUD - 30 User License                          G230830|0002AO-GEN
          Texas A&M University, Chemical Engineering                DC11194
          C:\GAMS\45\gamslice.txt
          License Admin: Jeff Polasek, j-polasek@tamu.edu                  
          The maintenance period of the license expired on Jun 25, 2024
          Please contact GAMS or your distributor for further information
Processor information: 1 socket(s), 16 core(s), and 24 thread(s) available
GAMS 45.7.0   Copyright (C) 1987-2024 

In [53]:
m.sf.pprint()

sf : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :     0 : 0.799999999999955 :  None : False : False : NonNegativeReals


In [54]:
m.d0.pprint()

d0 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   5.0 :     5 : False : False :  Reals


In [55]:
m.d1.pprint()

d1 : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :     0 : 1.146060574105673 :     5 : False : False :  Reals


In [56]:
m.region_binaries.pprint()

region_binaries : Size=13122, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214